In [8]:
from Models import FCN
import os
import argparse
import openslide
import numpy as np
from keras.utils import multi_gpu_model
import time
import ScanNet2_deploy 
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import morphology, measure, color, io, filters
import tensorflow as tf
from preprocess.utils import openwholeslide, filter_ch, overlap1D, intersect, get_sample_mask

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # 不全部占满显存, 按需分配
session = tf.Session(config=config)
data_path = '/home/data/ACDC/'
def pre(m, id, x, y, n=83, alpha=4):
    ws_path = data_path + f'test/{id}.tif'
    # print(ws_path)

    strides = int(32 / alpha)
    size = (n - 1) * 32 + 244

    reader = openslide.OpenSlide(ws_path)
    img = reader.read_region((y, x), 0, (size + strides * (alpha - 1), size + strides * (alpha - 1))).convert('RGB')
    img = np.asarray(img)
    # print("readed..",time.time()-start)

    # import cv2
    # cv2.imwrite('prediction/150/test.bmp',img)

    img = img.astype('float32')
    img /= 255.
    _input = []
    prediction = np.zeros((n * alpha, n * alpha))
    for iter in range(alpha * alpha):
        i = int(iter / alpha)
        j = int(iter % alpha)
        _input.append(img[i * strides:i * strides + size, j * strides:j * strides + size, :])
    _input = np.asarray(_input)
    assert _input.shape == (alpha * alpha, size, size, 3)
    _prediction = m.predict(_input, batch_size=1)[0, :, :, 1]
    return _prediction
def merge_bbox(bboxes: list):
    """
    整合bbox, 将有相交的整合成更大的bbox
    :param bboxes:
    :return:
    """
    for idx in range(len(bboxes)):
        current_bbox = bboxes[idx]
        for next_idx in range(idx + 1, len(bboxes)):
            other_bbox = bboxes[next_idx]
            if intersect(current_bbox, other_bbox):
                minr = min(current_bbox[0], other_bbox[0])
                minc = min(current_bbox[1], other_bbox[1])
                maxr = max(current_bbox[2], other_bbox[2])
                maxc = max(current_bbox[3], other_bbox[3])
                bboxes.pop(idx)
                bboxes.pop(next_idx - 1)
                bboxes.append((minr, minc, maxr, maxc))
                return merge_bbox(bboxes)
    return bboxes

In [11]:
def predict_mask(id):
    load_weights = 'logs/017/model_09-0.82.hdf5'
    n=83
    flag='pre'
    alpha=1
    if not os.path.exists(f'test/{id}/'):
        os.mkdir(f'test/{id}')
    size = (n - 1) * 32 + 244
    DPT_strides = 32 * n
    start = time.time()
    m = ScanNet2_deploy.GetModel(flag='pre', size=size)
    m.load_weights(load_weights, by_name=True)

    print("loaded..", time.time() - start)
    tif_path = f'/home/data/ACDC/test/{id}.tif'
    slide_reader, levels, dims = openwholeslide(tif_path)
    w, h = dims[-2]
    scale_factor_row, scale_factor_col = float(
        dims[0][1]) / h, float(dims[0][0]) / w
    thumbnail = np.asarray(slide_reader.get_thumbnail((w, h)).convert('RGB'))
    sample_mask = get_sample_mask(thumbnail)
    label_image = measure.label(sample_mask)
    sample_bbox = []
    for region in measure.regionprops(label_image):
        if region.area > 10:ubuntu
            sample_bbox.append(region.bbox)
    sample_bbox = merge_bbox(sample_bbox)
    print(scale_factor_row, scale_factor_col)
    print(sample_bbox)
    for iter, region_bbox in enumerate(sample_bbox):
            print('*'*30)
            print("REGION BBOX:", region_bbox)

            xmin = int(region_bbox[0] * scale_factor_row)
            ymin = int(region_bbox[1] * scale_factor_col)
            xmax = int(region_bbox[2] * scale_factor_row)
            ymax = int(region_bbox[3] * scale_factor_col)
            col_count = int((ymax - ymin-int(32 * (alpha - 1) / alpha)-size) / DPT_strides)+2
            row_count = int((xmax - xmin-int(32 * (alpha - 1) / alpha)-size) / DPT_strides)+2

            r = 0
            for r in range(row_count):
                for c in range(col_count):
                    #print(xmin + r * DPT_strides, ymin + c * DPT_strides)
                    if c == 0:
                        _prediction = pre(m, id, r * DPT_strides + xmin, c * DPT_strides + ymin, alpha=alpha)
                        #print(_prediction.shape)
                        assert(_prediction.shape==(n,n))
                    else:
                        tmp = pre(m, id, r * DPT_strides + xmin, c * DPT_strides + ymin, alpha=alpha)
                        _prediction = np.hstack((_prediction, tmp))
                if r == 0:
                    prediction = _prediction.copy()
                else:
                    prediction = np.vstack((prediction, _prediction))
            xmax = xmin+size+DPT_strides*(row_count-1)        
            ymax = ymin+size+DPT_strides*(col_count-1)
            np.save(f'test/{id}/{xmin}_{ymin}_{xmax}_{ymax}.npy', prediction)
            print('*'*30)

In [12]:
for i in range(1, 51):
    predict_mask(i)

/home/jcwang/ACDC1/ScanNet2_deploy.py:51: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input=input, output=x)


loaded.. 0.8752014636993408
loading 1.tif
1.tif loaded successfully
32.03463855421687 32.00075872534143
[(4, 1133, 132, 1315), (147, 816, 189, 847), (222, 953, 319, 1055), (226, 841, 251, 866), (108, 867, 172, 1049), (183, 72, 389, 304), (188, 426, 389, 743), (477, 208, 598, 301)]
******************************
REGION BBOX: (4, 1133, 132, 1315)
******************************
******************************
REGION BBOX: (147, 816, 189, 847)
******************************
******************************
REGION BBOX: (222, 953, 319, 1055)
******************************
******************************
REGION BBOX: (226, 841, 251, 866)
******************************
******************************
REGION BBOX: (108, 867, 172, 1049)
******************************
******************************
REGION BBOX: (183, 72, 389, 304)
******************************
******************************
REGION BBOX: (188, 426, 389, 743)
******************************
******************************
REGION BBOX: (

******************************
******************************
REGION BBOX: (823, 665, 951, 766)
******************************
******************************
REGION BBOX: (815, 27, 1247, 438)
******************************
loaded.. 1.8704640865325928
loading 20.tif
20.tif loaded successfully
16.002803738317755 16.015200868621065
[(814, 516, 1048, 765), (837, 154, 891, 186), (864, 128, 882, 150), (200, 588, 802, 904), (15, 0, 556, 565)]
******************************
REGION BBOX: (814, 516, 1048, 765)
******************************
******************************
REGION BBOX: (837, 154, 891, 186)
******************************
******************************
REGION BBOX: (864, 128, 882, 150)
******************************
******************************
REGION BBOX: (200, 588, 802, 904)
******************************
******************************
REGION BBOX: (15, 0, 556, 565)
******************************
loaded.. 1.9004240036010742
loading 21.tif
21.tif loaded successfully
16.001792114

******************************
******************************
REGION BBOX: (51, 83, 307, 479)
******************************
loaded.. 3.2104923725128174
loading 39.tif
39.tif loaded successfully
8.003503854239664 8.004019292604502
[(1005, 1046, 1427, 1240), (12, 99, 1291, 992)]
******************************
REGION BBOX: (1005, 1046, 1427, 1240)
******************************
******************************
REGION BBOX: (12, 99, 1291, 992)
******************************
loaded.. 3.5454180240631104
loading 40.tif
40.tif loaded successfully
64.0294345468629 64.02922940655448
[(152, 0, 1291, 1129)]
******************************
REGION BBOX: (152, 0, 1291, 1129)
******************************
loaded.. 3.0044713020324707
loading 41.tif
41.tif loaded successfully
32.010385756676556 32.0
[(45, 490, 250, 744), (211, 16, 406, 169), (276, 207, 674, 522)]
******************************
REGION BBOX: (45, 490, 250, 744)
******************************
******************************
REGION BBOX: (211